# Timeline Visualisation

This notebook reconstructs how events flowed through the worker system and visualise the processing timeline.

### Libraries

In [21]:
import pandas as pd
import plotly.express as px
import numpy as np

### Macro Definitions

In [22]:
FILENAME="03_worker_logs.csv"
NUM_EVENTS_PLOTTED=20 # For better readability, restricting the number of events that get plotted on the Gantt Plot.

### Load data

In [23]:
df = pd.read_csv(FILENAME, parse_dates=['start_time', 'end_time'])

## Overlapping processing windows

In [24]:
# Finding race conditions or overlapping processing windows
try:
    df.columns = df.columns.str.strip()
    df['start_time'] = pd.to_datetime(df['start_time'], format='ISO8601')
    df['end_time'] = pd.to_datetime(df['end_time'], format='ISO8601')
except Exception as e:
    print(f"Error processing datetime columns: {e}")

In [25]:
def assign_overlap_groups(event_df):
    
    if event_df.empty:
        return event_df
    
    global global_group_id
    
    try:
        start_times = event_df['start_time'].values
        end_times = event_df['end_time'].values
        n = len(start_times)
        
        overlap_groups = np.zeros(n, dtype=int)
        current_end_time = pd.Timestamp.min
        
        for i in range(n):
            if start_times[i] <= current_end_time:
                overlap_groups[i] = global_group_id
                current_end_time = max(current_end_time, end_times[i])
            else:
                global_group_id += 1
                overlap_groups[i] = global_group_id
                current_end_time = end_times[i]
        
        event_df['overlap_group'] = overlap_groups
        
    except Exception as e:
        print(f"Error assigning overlap groups for event_df: {e}")
        event_df['overlap_group'] = -1
    
    return event_df

In [26]:
try:
    overlap_df = df.sort_values(by=['event_id', 'start_time']).reset_index(drop=True)
    overlap_df['overlap_group'] = -1
    global_group_id = 0

    overlap_df = overlap_df.groupby('event_id', group_keys=False).apply(assign_overlap_groups).reset_index(drop=True)
    
except Exception as e:
    print(f"Error during overlap detection: {e}")

C:\Users\DELL\AppData\Local\Temp\ipykernel_29268\2781186518.py:6: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [27]:
try:
    overlap_counts = overlap_df.groupby('overlap_group').size()
    true_overlaps = overlap_counts[overlap_counts > 1]
    num_overlaps = len(true_overlaps)

    print(f"Overlapping processing windows for the same event between same or different workers:\n")
    print(f"Number of overlapping events: {num_overlaps}")

    overlaps_rows = overlap_df[overlap_df['overlap_group'].isin(true_overlaps.index)].reset_index(drop=True)
    print(overlaps_rows[['event_id', 'worker_id', 'start_time', 'end_time']])

except Exception as e:
    print(f"Error analyzing overlap results: {e}")


Overlapping processing windows for the same event between same or different workers:

Number of overlapping events: 250
        event_id worker_id              start_time                end_time
0    evt_0001162  worker-3 2025-07-12 09:15:05.810 2025-07-12 09:15:05.821
1    evt_0001162  worker-2 2025-07-12 09:15:05.810 2025-07-12 09:15:05.817
2    evt_0001289  worker-2 2025-07-12 09:15:06.445 2025-07-12 09:15:06.459
3    evt_0001289  worker-1 2025-07-12 09:15:06.445 2025-07-12 09:15:06.457
4    evt_0001752  worker-3 2025-07-12 09:15:08.760 2025-07-12 09:15:08.774
..           ...       ...                     ...                     ...
495  evt_0069129  worker-3 2025-07-12 09:20:45.645 2025-07-12 09:20:45.656
496  evt_0069895  worker-2 2025-07-12 09:20:49.475 2025-07-12 09:20:49.480
497  evt_0069895  worker-1 2025-07-12 09:20:49.475 2025-07-12 09:20:49.478
498  evt_0069915  worker-2 2025-07-12 09:20:49.575 2025-07-12 09:20:49.590
499  evt_0069915  worker-3 2025-07-12 09:20:49.575 2025

## Retried jobs

In [28]:
# Identifying Retried jobs
try:
    retried_df = df.sort_values(by=['event_id', 'start_time']).reset_index(drop=True)
    retried_df['prev_end_time'] = retried_df.groupby('event_id')['end_time'].shift()

    retried_df['is_retry'] = (retried_df['start_time'] > retried_df['prev_end_time'])

    retried_df['is_retry'] = retried_df['is_retry'].fillna(False)

    num_retries = retried_df['is_retry'].sum()

    print(f"Retried jobs that is non-overlapping attempts for same event by the same or different worker:\n")
    print(f"Number of retried jobs: {int(num_retries)}\n")

    retried_rows = retried_df[retried_df['is_retry']]
    print("Retried jobs:")
    if retried_rows.empty:
        print("Empty Dataframe")
    else:
        print(retried_rows[['event_id', 'worker_id', 'start_time', 'end_time']])
    
except Exception as e:
    print(f"Error identifying retried jobs: {e}")

Retried jobs that is non-overlapping attempts for same event by the same or different worker:

Number of retried jobs: 0

Retried jobs:
Empty Dataframe


## Events that failed

In [29]:
# Identifying failed events
try:
    failed_events = df[df['status'] != 'success']

    print(f"\nAll the failed events (that is job status is not success):\n")
    print(f"Number of Failed Events : {len(failed_events)}\n")
    print(f"Failed Events:")
    print(failed_events[['event_id', 'worker_id', 'start_time', 'end_time', 'status']])
    
except Exception as e:
    print(f"Error identifying failed jobs: {e}")


All the failed events (that is job status is not success):

Number of Failed Events : 100

Failed Events:
          event_id worker_id              start_time end_time   status
247    evt_0000247  worker-3 2025-07-12 09:15:01.235      NaT  timeout
1204   evt_0001203  worker-1 2025-07-12 09:15:06.015      NaT  timeout
1745   evt_0001743  worker-2 2025-07-12 09:15:08.715      NaT  timeout
1887   evt_0001884  worker-1 2025-07-12 09:15:09.420      NaT  timeout
2047   evt_0002043  worker-2 2025-07-12 09:15:10.215      NaT  timeout
...            ...       ...                     ...      ...      ...
65102  evt_0064868  worker-1 2025-07-12 09:20:24.340      NaT  timeout
65376  evt_0065142  worker-3 2025-07-12 09:20:25.710      NaT  timeout
66176  evt_0065938  worker-2 2025-07-12 09:20:29.690      NaT  timeout
67220  evt_0066980  worker-2 2025-07-12 09:20:34.900      NaT  timeout
70018  evt_0069770  worker-3 2025-07-12 09:20:48.850      NaT  timeout

[100 rows x 5 columns]


## Gantt chart per worker

In [30]:
# Create the Gantt plot
try:
    # Selecting first 20 event_ids for plotting for better readabilty 
    selected_events = df['event_id'].dropna()[:NUM_EVENTS_PLOTTED]
    df_subset = df[df['event_id'].isin(selected_events)].copy()

    fig = px.timeline(
        df_subset,
        x_start="start_time",
        x_end="end_time",
        y="worker_id",
        color="status", 
        text="event_id",
        title="Worker Event Timeline (First 20 Events)"
    )

    fig.update_yaxes(autorange="reversed")
    fig.update_traces(
        textposition="inside",
        insidetextanchor="start",
        marker_line_width=0.5,
        marker_line_color="black"
    )

    fig.update_layout(
        xaxis_title="Time",
        yaxis_title="Worker ID",
        font=dict(size=12),
        height=700,
        margin=dict(l=80, r=50, b=50, t=80),
        title_font_size=22,
        legend_title_text="Status"
    )

    fig.show()

except Exception as e:
    print(f"Error generating Gantt plot: {e}")
